In [1]:
import lmdb
from bigearthnet_patch_interface.s2_interface import BigEarthNet_S2_Patch
from pathlib import Path
import importlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import Tensor, cat, stack
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

from sklearn.metrics import accuracy_score

### Prepare Data

In [12]:
import os
from dotenv import load_dotenv

import dataset_class as ds_class
importlib.reload(ds_class)

load_dotenv('./.env')

PATH_TO_MODELS = os.environ.get("PATH_TO_MODELS")
BEN_LMDB_PATH = os.environ.get("BEN_LMDB_PATH")
TEST_CSV_FILE = os.environ.get("TEST_CSV")

assert Path(BEN_LMDB_PATH).exists()
assert Path(TEST_CSV_FILE).exists()
assert Path(PATH_TO_MODELS).exists()
assert os.path.isdir(PATH_TO_MODELS)

env = lmdb.open(BEN_LMDB_PATH, readonly=True, readahead=False, lock=False)
txn = env.begin()
cur = txn.cursor()

test_ds = ds_class.BenDataset(TEST_CSV_FILE, BEN_LMDB_PATH, size=50)
test_loader = DataLoader(test_ds, batch_size=25, shuffle=True)

### Prepare Model

In [24]:
import training_utils as train_utils
importlib.reload(train_utils)

model_dir = '05-27_1855-ConvMx-256-8'
model_name = 'ConvMx-256-8.pt'

model = train_utils.load_complete_model(PATH_TO_MODELS + f'/{model_dir}/{model_name}')
model.eval()

ConvMixer(
  (patch_embedding): Sequential(
    (0): Conv2d(10, 256, kernel_size=(7, 7), stride=(7, 7))
    (1): GELU()
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ConvMixerLayers): ModuleList(
    (0): ConvMixerLayer(
      (depthwise_conv): Sequential(
        (0): Conv2d(256, 256, kernel_size=(9, 9), stride=(1, 1), padding=same, groups=256)
        (1): GELU()
        (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (pointwise_conv): Sequential(
        (0): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): GELU()
        (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ConvMixerLayer(
      (depthwise_conv): Sequential(
        (0): Conv2d(256, 256, kernel_size=(9, 9), stride=(1, 1), padding=same, groups=256)
        (1): GELU()
        (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_runn

### Test

In [25]:
acc_sum = 0.0

for X, y_true in test_loader:
    
    with torch.no_grad():
        outputs_sig = torch.sigmoid(model(X))
        y_pred = torch.round(outputs_sig)
        acc_sum += train_utils.get_accuracy_for_batch(y_true, y_pred)

        
print(f'test_acc={acc_sum / len(test_loader)}')
        

test_acc=0.5578947368421054
